In [139]:
import pandas as pd
import numpy as np

In [140]:
incidents_dataframe = pd.read_csv("../featured_datasets/Taffic_Incidents_Braga_Featured.csv",",")
incidents_dataframe

,description,from_road,to_road,incident_category_desc,magnitude_of_delay_desc,length_in_meters,delay_in_seconds,latitude,longitude,Day of year,week_or_not,Hour,Minute
0,queuing traffic,Quinteiro (N103),Braga-Circular (Ferreiros) (N103),Jam,2,0.083241,111,41.537361,-8.452289,15,1,19,5
1,queuing traffic,Rua Manuel Almeida Passos (N14),Rua Da Quinta De Santa Maria (N14),Jam,2,0.137673,210,41.531712,-8.440025,15,1,19,5
2,queuing traffic,Braga-Circular (N101),Avenida General Norton De Matos (N101),Jam,2,0.083241,184,41.559666,-8.418733,15,1,19,5
3,stationary traffic,Avenida General Norton De Matos (N101),Braga-Circular (N101),Jam,3,0.042659,232,41.557526,-8.417557,15,1,19,5
4,stationary traffic,Cm1327 (N201),N201 (Rua De Cima) (N201),Jam,3,0.026870,235,41.558880,-8.445529,15,1,19,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83334,stationary traffic,Rua Monsenhor Airosa (N309) / Rua Conselheiro ...,Rua Conselheiro Lobato / Rua Monsenhor Airosa ...,Jam,3,0.019806,178,41.544969,-8.420512,365,1,18,46
83335,stationary traffic,Avenida da Liberdade (N101) / Avenida João XXI...,Rua de Diu,Jam,3,0.017036,182,41.547453,-8.420823,365,1,19,26
83336,stationary traffic,Rua Monsenhor Airosa (N309) / Rua Conselheiro ...,Rua Conselheiro Lobato / Rua Monsenhor Airosa ...,Jam,3,0.019806,208,41.544969,-8.420512,365,1,20,46
83337,stationary traffic,Rua Monsenhor Airosa (N309) / Rua Conselheiro ...,Rua Conselheiro Lobato / Rua Monsenhor Airosa ...,Jam,3,0.019806,209,41.544969,-8.420512,365,1,21,6


In [141]:
incidents = incidents_dataframe.to_numpy()

In [142]:
#colunas 11 e 12 apresentam a latitude e longitutde
incidents[:,7:9]

array([[41.53736114501957, -8.452288627624513],
       [41.53171157836915, -8.440025329589844],
       [41.55966567993163, -8.418732643127441],
       ...,
       [41.5449688, -8.4205123],
       [41.5449688, -8.4205123],
       [41.551557700000004, -8.4172319]], dtype=object)

> A fórmula de Haversine é uma importante equação usada em navegação, fornecendo distâncias entre dois pontos de uma esfera a partir de suas latitudes e longitudes. 

> É um caso especial de uma fórmula mais geral de trigonometria esférica, a lei dos Haversines, relacionando os lados a ângulos de uma esfera "triangular".

In [143]:
import math

def haversine(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [144]:
#Latitude e longitude
#Avenida Central, esta rua só mexe verticalmente como podemos observar pela longitude
(ll_min_ac) = (41.551,-8.418)
(ll_max_ac) = (41.551,-8.423)

#Avenida Liberdade
(ll_min_al) = (41.551,-8.423)
(ll_max_al) = (41.542,-8.418)

#Rua de Caires, só mexe horizontalmente
(ll_min_rc) = (41.548,-8.433)
(ll_max_rc) = (41.544,-8.433)

In [145]:
haversine(ll_min_al,ll_max_al)

1084.1186962075008

In [146]:
#arrays representates das distâncias de cada rua aos respetivos incididentes
ac_distances = np.zeros(shape=(len(incidents)))
al_distances = np.zeros(shape=(len(incidents)))
rc_distances = np.zeros(shape=(len(incidents)))

In [147]:
for i in range(0,len(incidents)):
    d = incidents[i][7:9]
    #quero a menor distancia, neste caso ao início da estrada ou no fim, quero sempre a menor distancia
    
    #também posso meter a loalização do centro da estrada caso seja pertinente
    for tmp in range(0,3):
        if(tmp == 0):
            x = ac_distances
            distance1 = haversine(d,ll_min_ac)
            distance2 = haversine(d,ll_max_ac)
        elif(tmp==1):
            x = al_distances
            distance1 = haversine(d,ll_min_al)
            distance2 = haversine(d,ll_max_al)
        elif(tmp==2):
            x = rc_distances
            distance1 = haversine(d,ll_min_rc)
            distance2 = haversine(d,ll_max_rc)
    
        if(distance1 < distance2):
            x[i] = distance1
        else:
            x[i] = distance2

In [148]:
ac_dataframe = pd.DataFrame({'AvenidaCentral': ac_distances})
al_dataframe = pd.DataFrame({'AvenidaLiberdade': al_distances})
rc_dataframe = pd.DataFrame({'RuaDeCaires': rc_distances})

In [149]:
al_dataframe

,AvenidaLiberdade
0,2871.589353
1,2161.513827
2,1027.212214
3,855.618311
4,2069.868805
...,...
83334,390.869095
83335,434.184575
83336,390.869095
83337,390.869095


In [150]:
incidents_dataframe = pd.concat([incidents_dataframe,ac_dataframe,al_dataframe,rc_dataframe],axis=1)

In [151]:
incidents_dataframe.to_csv(r'../featured_datasets/incidents_with_distances.csv',index=False)

In [152]:
incidents_dataframe

,description,from_road,to_road,incident_category_desc,magnitude_of_delay_desc,length_in_meters,delay_in_seconds,latitude,longitude,Day of year,week_or_not,Hour,Minute,AvenidaCentral,AvenidaLiberdade,RuaDeCaires
0,queuing traffic,Quinteiro (N103),Braga-Circular (Ferreiros) (N103),Jam,2,0.083241,111,41.537361,-8.452289,15,1,19,5,2871.589353,2871.589353,1767.444236
1,queuing traffic,Rua Manuel Almeida Passos (N14),Rua Da Quinta De Santa Maria (N14),Jam,2,0.137673,210,41.531712,-8.440025,15,1,19,5,2571.299699,2161.513827,1486.684810
2,queuing traffic,Braga-Circular (N101),Avenida General Norton De Matos (N101),Jam,2,0.083241,184,41.559666,-8.418733,15,1,19,5,965.778946,1027.212214,1758.926643
3,stationary traffic,Avenida General Norton De Matos (N101),Braga-Circular (N101),Jam,3,0.042659,232,41.557526,-8.417557,15,1,19,5,726.759470,855.618311,1665.794200
4,stationary traffic,Cm1327 (N201),N201 (Rua De Cima) (N201),Jam,3,0.026870,235,41.558880,-8.445529,15,1,19,5,2069.868805,2069.868805,1597.478316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83334,stationary traffic,Rua Monsenhor Airosa (N309) / Rua Conselheiro ...,Rua Conselheiro Lobato / Rua Monsenhor Airosa ...,Jam,3,0.019806,178,41.544969,-8.420512,365,1,18,46,702.063438,390.869095,1045.125415
83335,stationary traffic,Avenida da Liberdade (N101) / Avenida João XXI...,Rua de Diu,Jam,3,0.017036,182,41.547453,-8.420823,365,1,19,26,434.184575,434.184575,1015.434442
83336,stationary traffic,Rua Monsenhor Airosa (N309) / Rua Conselheiro ...,Rua Conselheiro Lobato / Rua Monsenhor Airosa ...,Jam,3,0.019806,208,41.544969,-8.420512,365,1,20,46,702.063438,390.869095,1045.125415
83337,stationary traffic,Rua Monsenhor Airosa (N309) / Rua Conselheiro ...,Rua Conselheiro Lobato / Rua Monsenhor Airosa ...,Jam,3,0.019806,209,41.544969,-8.420512,365,1,21,6,702.063438,390.869095,1045.125415


In [153]:
ac_grau = np.zeros(shape=(len(incidents)))
al_grau = np.zeros(shape=(len(incidents)))
rc_grau = np.zeros(shape=(len(incidents)))    

icd = incidents_dataframe.to_numpy()
for i in range(len(icd)):
    #1 sobre distancaia * lenght_in_meters * magnitude^2 
    dist_ac = 1/ np.sqrt(icd[i][-3])
    dist_al = 1/np.sqrt(icd[i][-2])
    dist_rc = 1/np.sqrt(icd[i][-1])
    
    magn = icd[i][5] ** 2
    
    lenght = icd[i][6]
    
    ac_grau[i] = dist_ac * magn * lenght
    al_grau[i] = dist_al * magn * lenght
    rc_grau[i] = dist_rc * magn * lenght

In [155]:
ac_dataframe_grau = pd.DataFrame({'AvenidaCentral_Grau': ac_grau})
al_dataframe_grau = pd.DataFrame({'AvenidaLiberdade_Grau': al_grau})
rc_dataframe_grau = pd.DataFrame({'RuaDeCaires_Grau': rc_grau})

In [156]:
incidents_dataframe = pd.concat([incidents_dataframe,ac_dataframe_grau,al_dataframe_grau,rc_dataframe_grau],axis=1)

In [157]:
incidents_dataframe.to_csv(r'../featured_datasets/incidents_with_distances.csv',index=False)